# Evaluation for the first model

In [ ]:
from pathlib import Path
import numpy as np
import torch 

In [ ]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '../')

In [ ]:
from utils.models.fully_connected import FullyConnected, get_loss_function

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_angle_datasets

In [ ]:
data_folder = Path("../../data/")
train_data, test_data = read_angle_datasets(data_folder, 0.9)

In [ ]:
input_shape, output_shape = train_data[0][0].shape[0], train_data[0][1].shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_data) + len(test_data)} data rows!")

## Load parameter, functions and dataloader

In [ ]:
import os
from dotenv import load_dotenv
from torch.utils.data import DataLoader

from utils.evaluation import compute_loss_on
from utils.file_io import define_dataloader_from_angle_dataset

In [ ]:
model_path = Path("../../models/fully_connected/").absolute()

## Load trail grid and best train loss

In [ ]:
import pandas as pd
import ast

In [ ]:
result_grid = pd.read_csv(model_path / "trail_grid_.csv")
result_grid.sort_values('loss').head(5)

In [ ]:
# Load best train loss
best_result = result_grid.iloc[result_grid['loss'].idxmin()].to_dict()
trail_id = best_result['trial_id']

print(f"Trail ID from the best run: {trail_id}")

## 1. Analysis: Compute test loss

In [ ]:
from utils.evaluation import compute_losses_from, compute_predictions

In [ ]:
_, _, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, best_result['config/batch_size'], split_size=0.95)

In [ ]:
model = FullyConnected(input_shape, ast.literal_eval(best_result["config/hidden_layers"]), output_shape, best_result["config/dropout"])
model.load_state_dict(torch.load(model_path / trail_id / "checkpoint.pt"))
model.eval()

In [ ]:
y, y_true = compute_predictions(test_dataloader, model, 'cpu')
test_losses = compute_losses_from(y, y_true, get_loss_function())
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")

## 2. Analysis: Trace animation

In [ ]:
from utils.visualization import create_trace_animation
from matplotlib import pyplot as plt
from IPython.display import HTML

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

In [ ]:
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())